In [1]:
!ls 

CS519 HW1.ipynb
hw1-random_output.py
hw1-validate.py
hw1.tex
income.dev.txt
income.test.blind
income.toy.txt
income.train.txt.5k


In [2]:
!cat income.train.txt.5k | cut -f 10 -d ',' | sort | uniq -c

   3749  <=50K
   1251  >50K


In [3]:
1251 / (1251 + 3749)

0.2502

In [4]:
!cat income.dev.txt | cut -f 10 -d ',' | sort | uniq -c

    764  <=50K
    236  >50K


In [5]:
236 / (236 + 764)

0.236

In [6]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort -nk1| head -1

17


sort: fflush failed: 'standard output'
sort: write error


In [7]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort -nk1| tail -1

90


In [8]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort -nk1| head -1

 1


sort: fflush failed: 'standard output'
sort: write error


In [9]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort -nk1| tail -1

 99


In [10]:
!cat income.train.txt.5k | cut -f 2 -d ',' | sort | uniq | wc -l

7


In [11]:
!cat income.train.txt.5k | cut -f 3 -d ',' | sort | uniq | wc -l

16


In [12]:
!cat income.train.txt.5k | cut -f 4 -d ',' | sort | uniq | wc -l

7


In [13]:
!cat income.train.txt.5k | cut -f 5 -d ',' | sort | uniq | wc -l

14


In [14]:
!cat income.train.txt.5k | cut -f 6 -d ',' | sort | uniq | wc -l

5


In [15]:
!cat income.train.txt.5k | cut -f 7 -d ',' | sort | uniq | wc -l

2


In [16]:
!cat income.train.txt.5k | cut -f 9 -d ',' | sort | uniq | wc -l

39


In [17]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort | uniq | wc -l

67


In [18]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort | uniq | wc -l

73


In [19]:
import numpy as np

In [20]:
import time

In [21]:
def binarization(file, mapping=None):
    # Intialize data array from file
    data = map(lambda s: s.strip().split(", "), open(file).readlines())
    
    # Initialize variables
    if mapping is None:
        mapping = {}
    categorical_data = []
    numerical_data = [[],[]]
    target_data = []
    min_v = [999, 999]
    max_v = [-1, -1]
    
    # Map categorical features and save numerical fields
    for row in data:
        new_row = []
        for j, x in enumerate(row):
            if j == 0:
                numerical_data[1].append(x)
                if int(x) < min_v[1]:
                    min_v[1] = int(x)
                if int(x) > max_v[1]:
                    max_v[1] = int(x)
                continue
            if j == 7: 
                numerical_data[0].append(x)
                if int(x) < min_v[0]:
                    min_v[0] = int(x)
                if int(x) > max_v[0]:
                    max_v[0] = int(x)
                continue
            if j == 9:
                target_data.append(x)
                continue
            feature = (j, x)
            if feature not in mapping:
                mapping[feature] = len(mapping)
            new_row.append(mapping[feature])
        categorical_data.append(new_row)
        
    # Binarize categorical data
    print(str(len(categorical_data)) + ', ' + str(len(mapping)) + ', ' + str(len(numerical_data)))
          
    binary_data = np.zeros((len(categorical_data), len(mapping) + len(numerical_data)))
          
    for i, row in enumerate(categorical_data):
        for x in row:
            binary_data[i][x] = 1
            
    # Add weighted numerical data to binary data
    for i, nv in enumerate(numerical_data):
        div = (max_v[i] - min_v[i]) / 2
        for j, value in enumerate(nv):
            binary_data[j][binary_data[0].size - 1 - i] = int(value) / div
    
    return [binary_data, mapping, target_data]
    

In [22]:
def euclidean(p1, p2):
    return np.linalg.norm(p1-p2)

In [23]:
def manhattan(p1, p2):
    return np.sum(abs(p1 - p2))

In [24]:
def k_NN(comp, arr, method, k):
    nn = []
    dist = []
    
    for j, data in enumerate(arr):
        
        # While nearest neighbor list does not have k elements
        if len(nn) < k:
            # If first element, add to nearest neighbor
            if len(nn) == 0:
                nn.append(j)
                dist.append(method(comp, arr[j]))
            
            # Else insert in ascending order to nearest neighbors
            else:
                for n in range(len(nn)):
                    if method(comp, arr[j]) < method(comp, arr[nn[n]]):
                        nn.insert(n, j)
                        dist.insert(n, method(comp, arr[j]))
                        break
                    elif n == len(nn) - 1:
                        nn.append(j)
                        dist.append(method(comp, arr[j]))
                        
        # Otherwise insert in ascending order and delete last element           
        else:
            for n in range(len(nn)):
                if method(comp, arr[j]) < method(comp, arr[nn[n]]):
                    nn.insert(n, j)
                    dist.insert(n, method(comp, arr[j]))
                    del nn[-1]
                    del dist[-1]
                    break
                    
    return [nn, dist]

In [25]:
def k_NN_predict(comp, bin_arr, method, k):
    
    # Map target possibilities
    mapping = {}
    for j, x in enumerate(bin_arr[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
    
    # Generate count for map from neighbors
    count = []
    for feature in mapping:
        count.append(0)
    
    nn_arr = k_NN(comp, bin_arr[0], method, k)
    
    for nn in nn_arr[0]:
        count[mapping[bin_arr[2][nn]]] += 1
    
    # Return prediction based on majority
    for feature in mapping:
        if count.index(max(count)) == mapping[feature]:
            return feature
    
    return

In [26]:
train = binarization('income.train.txt.5k')

5000, 90, 2


In [27]:
dev = binarization('income.dev.txt', train[1])

1000, 91, 2


In [28]:
train = binarization('income.train.txt.5k', dev[1])

5000, 91, 2


In [29]:
train[1] == dev[1]

True

In [30]:
len(dev[1])

91

In [31]:
len(train[1])

91

In [32]:
k_NN(dev[0][-1], train[0], euclidean, 5)

[[1010, 1713, 3769, 2003, 3698],
 [0.08262150731590351,
  0.2193395913868282,
  0.2737218598779718,
  0.34672196259217397,
  0.40068528287721883]]

In [33]:
k_NN(dev[0][-1], train[0], manhattan, 5)

[[1010, 1713, 3769, 3698, 2450],
 [0.09060751825090985,
  0.22759381962077296,
  0.2737218598779718,
  0.427144788151103,
  0.4741691620865267]]

In [34]:
!tail -1  income.dev.txt 

58, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [35]:
!sed -n 1011p income.train.txt.5k

55, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [36]:
!sed -n 1714p income.train.txt.5k

66, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [37]:
!sed -n 3770p income.train.txt.5k

58, Private, HS-grad, Widowed, Adm-clerical, White, Female, 27, United-States, <=50K


In [38]:
!sed -n 2004p income.train.txt.5k

68, Private, HS-grad, Widowed, Adm-clerical, White, Female, 30, United-States, <=50K


In [39]:
!sed -n 3699p income.train.txt.5k

59, Private, HS-grad, Widowed, Adm-clerical, White, Female, 60, United-States, <=50K


In [40]:
!sed -n 2451p income.train.txt.5k

75, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [41]:
k_NN(dev[0][-1], train[0], euclidean, 9)

[[1010, 1713, 3769, 2003, 3698, 2450, 3680, 681, 2731],
 [0.08262150731590351,
  0.2193395913868282,
  0.2737218598779718,
  0.34672196259217397,
  0.40068528287721883,
  0.465829450782897,
  0.48408415077534417,
  0.7671694487686503,
  1.4142386024417781]]

In [42]:
!sed -n 2451p income.train.txt.5k

75, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [43]:
!sed -n 3681p income.train.txt.5k

49, Private, HS-grad, Widowed, Adm-clerical, White, Female, 20, United-States, <=50K


In [44]:
!sed -n 682p income.train.txt.5k

30, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [45]:
!sed -n 2732p income.train.txt.5k

58, Private, HS-grad, Divorced, Adm-clerical, White, Female, 40, United-States, <=50K


In [46]:
k_NN(dev[0][-1], train[0], manhattan, 9)

[[1010, 1713, 3769, 3698, 2450, 2003, 3680, 681, 2731],
 [0.09060751825090985,
  0.22759381962077296,
  0.2737218598779718,
  0.427144788151103,
  0.4741691620865267,
  0.4864699728217794,
  0.6631543452008679,
  0.775539025100225,
  2.008415737428992]]

In [47]:
!sed -n 2004p income.train.txt.5k

68, Private, HS-grad, Widowed, Adm-clerical, White, Female, 30, United-States, <=50K


In [48]:
!sed -n 3681p income.train.txt.5k

49, Private, HS-grad, Widowed, Adm-clerical, White, Female, 20, United-States, <=50K


In [49]:
!sed -n 682p income.train.txt.5k

30, Private, HS-grad, Widowed, Adm-clerical, White, Female, 40, United-States, <=50K


In [50]:
!sed -n 2732p income.train.txt.5k

58, Private, HS-grad, Divorced, Adm-clerical, White, Female, 40, United-States, <=50K


In [51]:
k_NN_predict(dev[0][-1], train, euclidean, 5)

'<=50K'

In [52]:
def v_euclidean(arr, comp):
    return np.linalg.norm(arr - comp, axis=1)

In [53]:
def v_manhattan(arr,comp):
    return np.sum(abs(arr - comp), axis=1)

In [54]:
def v_k_NN(comp, arr, method, k):
    m_arr = method(arr, comp)
    nn_arr = np.argpartition(m_arr,k)[:k]
    dist = []
    
    for nn in nn_arr:
        dist.append(m_arr[nn])
        
    return [nn_arr, dist]

In [55]:
def v_k_NN_predict(comp, bin_arr, method, k, tar_map):
        
    # Map target possibilities
    mapping = tar_map
    
    
    # Generate count for map from neighbors
    count = []
    for feature in mapping:
        count.append(0)
    
    nn_arr = v_k_NN(comp, bin_arr[0], method, k)
    
    for nn in nn_arr[0]:
        count[mapping[bin_arr[2][nn]]] += 1
    
    # Return prediction based on majority
    for feature in mapping:
        if count.index(max(count)) == mapping[feature]:
            return [feature, mapping]

In [56]:
v_k_NN(dev[0][-1], train[0], v_euclidean, 5)

[array([1713, 1010, 3769, 2003, 3698], dtype=int64),
 [0.2193395913868282,
  0.08262150731590351,
  0.2737218598779718,
  0.34672196259217397,
  0.40068528287721883]]

In [57]:
k_set = [1, 3, 5, 7, 9, 99, 999]

In [58]:
def error_rate(k_set, method, test_set):
    output = []
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
                
    for k in k_set:
        print(k)
        correct = 0
        p_arr = None
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train, method, k, mapping)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)
                    
            # Update error and positive rate
            if p[0] == test_set[2][i]:
                correct += 1
            
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([correct , p_arr])
        output.append([correct / len(test_set[0]), p_arr])
        
    return output

In [59]:
s1 = time.time()

In [60]:
error_rate(k_set, v_euclidean, dev)

1
[759, [{'<=50K': 0, '>50K': 1}, [717, 283]]]
3
[804, [{'<=50K': 0, '>50K': 1}, [740, 260]]]
5
[822, [{'<=50K': 0, '>50K': 1}, [754, 246]]]
7
[834, [{'<=50K': 0, '>50K': 1}, [760, 240]]]
9
[842, [{'<=50K': 0, '>50K': 1}, [774, 226]]]
99
[843, [{'<=50K': 0, '>50K': 1}, [805, 195]]]
999
[822, [{'<=50K': 0, '>50K': 1}, [888, 112]]]


[[0.759, [{'<=50K': 0, '>50K': 1}, [717, 283]]],
 [0.804, [{'<=50K': 0, '>50K': 1}, [740, 260]]],
 [0.822, [{'<=50K': 0, '>50K': 1}, [754, 246]]],
 [0.834, [{'<=50K': 0, '>50K': 1}, [760, 240]]],
 [0.842, [{'<=50K': 0, '>50K': 1}, [774, 226]]],
 [0.843, [{'<=50K': 0, '>50K': 1}, [805, 195]]],
 [0.822, [{'<=50K': 0, '>50K': 1}, [888, 112]]]]

In [61]:
e1 = time.time()

In [62]:
e1 - s1

29.258525848388672

In [63]:
s2 = time.time()

In [64]:
error_rate(k_set, v_euclidean, train)

1
[4924, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]]
3
[4433, [{'<=50K': 0, '>50K': 1}, [3808, 1192]]]
5
[4313, [{'<=50K': 0, '>50K': 1}, [3800, 1200]]]
7
[4276, [{'<=50K': 0, '>50K': 1}, [3807, 1193]]]
9
[4229, [{'<=50K': 0, '>50K': 1}, [3816, 1184]]]
99
[4107, [{'<=50K': 0, '>50K': 1}, [4018, 982]]]
999
[3989, [{'<=50K': 0, '>50K': 1}, [4452, 548]]]


[[0.9848, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]],
 [0.8866, [{'<=50K': 0, '>50K': 1}, [3808, 1192]]],
 [0.8626, [{'<=50K': 0, '>50K': 1}, [3800, 1200]]],
 [0.8552, [{'<=50K': 0, '>50K': 1}, [3807, 1193]]],
 [0.8458, [{'<=50K': 0, '>50K': 1}, [3816, 1184]]],
 [0.8214, [{'<=50K': 0, '>50K': 1}, [4018, 982]]],
 [0.7978, [{'<=50K': 0, '>50K': 1}, [4452, 548]]]]

In [65]:
e2 = time.time()

In [66]:
e2 - s2

150.72938299179077

In [67]:
s3 = time.time()

In [68]:
error_rate(k_set, v_manhattan, dev)

1
[764, [{'<=50K': 0, '>50K': 1}, [726, 274]]]
3
[803, [{'<=50K': 0, '>50K': 1}, [745, 255]]]
5
[827, [{'<=50K': 0, '>50K': 1}, [751, 249]]]
7
[835, [{'<=50K': 0, '>50K': 1}, [761, 239]]]
9
[840, [{'<=50K': 0, '>50K': 1}, [776, 224]]]
99
[844, [{'<=50K': 0, '>50K': 1}, [808, 192]]]
999
[821, [{'<=50K': 0, '>50K': 1}, [891, 109]]]


[[0.764, [{'<=50K': 0, '>50K': 1}, [726, 274]]],
 [0.803, [{'<=50K': 0, '>50K': 1}, [745, 255]]],
 [0.827, [{'<=50K': 0, '>50K': 1}, [751, 249]]],
 [0.835, [{'<=50K': 0, '>50K': 1}, [761, 239]]],
 [0.84, [{'<=50K': 0, '>50K': 1}, [776, 224]]],
 [0.844, [{'<=50K': 0, '>50K': 1}, [808, 192]]],
 [0.821, [{'<=50K': 0, '>50K': 1}, [891, 109]]]]

In [69]:
e3 = time.time()

In [70]:
e3 - s3

31.321776628494263

In [71]:
s4 = time.time()

In [72]:
error_rate(k_set, v_manhattan, train)

1
[4924, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]]
3
[4422, [{'<=50K': 0, '>50K': 1}, [3811, 1189]]]
5
[4298, [{'<=50K': 0, '>50K': 1}, [3805, 1195]]]
7
[4268, [{'<=50K': 0, '>50K': 1}, [3805, 1195]]]
9
[4228, [{'<=50K': 0, '>50K': 1}, [3817, 1183]]]
99
[4098, [{'<=50K': 0, '>50K': 1}, [4017, 983]]]
999
[3989, [{'<=50K': 0, '>50K': 1}, [4480, 520]]]


[[0.9848, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]],
 [0.8844, [{'<=50K': 0, '>50K': 1}, [3811, 1189]]],
 [0.8596, [{'<=50K': 0, '>50K': 1}, [3805, 1195]]],
 [0.8536, [{'<=50K': 0, '>50K': 1}, [3805, 1195]]],
 [0.8456, [{'<=50K': 0, '>50K': 1}, [3817, 1183]]],
 [0.8196, [{'<=50K': 0, '>50K': 1}, [4017, 983]]],
 [0.7978, [{'<=50K': 0, '>50K': 1}, [4480, 520]]]]

In [73]:
e4 = time.time()

In [74]:
e4 - s4

148.79366993904114

In [75]:
def binarize_all(file, mapping=None):
    # Intialize data array from file
    data = map(lambda s: s.strip().split(", "), open(file).readlines())
    
    # Initialize variables
    if mapping is None:
        mapping = {}
    categorical_data = []
    target_data = []
    min_v = [999, 999]
    max_v = [-1, -1]
    
    # Map categorical features and save numerical fields
    for row in data:
        new_row = []
        for j, x in enumerate(row):
            if j == 9:
                target_data.append(x)
                continue
            feature = (j, x)
            if feature not in mapping:
                mapping[feature] = len(mapping)
            new_row.append(mapping[feature])
        categorical_data.append(new_row)
        
    # Binarize categorical data
    print(str(len(categorical_data)) + ', ' + str(len(mapping)))
          
    binary_data = np.zeros((len(categorical_data), len(mapping)))
          
    for i, row in enumerate(categorical_data):
        for x in row:
            binary_data[i][x] = 1
    
    return [binary_data, mapping, target_data]

In [76]:
def error_rate_all(k_set, method, test_set):
    output = []
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
                
    for k in k_set:
        print(k)
        correct = 0
        p_arr = None
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train_all, method, k, mapping)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)
                    
            # Update error and positive rate
            if p[0] == test_set[2][i]:
                correct += 1
            
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([correct , p_arr])
        output.append([correct / len(test_set[0]), p_arr])
        
    return output

In [77]:
train_all = binarize_all('income.train.txt.5k')

5000, 230


In [78]:
dev_all = binarize_all('income.dev.txt', train_all[1])

1000, 233


In [79]:
train_all = binarize_all('income.train.txt.5k', dev_all[1])

5000, 233


In [80]:
s5 = time.time()

In [81]:
error_rate_all(k_set, v_euclidean, dev_all)

1
[768, [{'<=50K': 0, '>50K': 1}, [752, 248]]]
3
[811, [{'<=50K': 0, '>50K': 1}, [775, 225]]]
5
[819, [{'<=50K': 0, '>50K': 1}, [787, 213]]]
7
[823, [{'<=50K': 0, '>50K': 1}, [795, 205]]]
9
[825, [{'<=50K': 0, '>50K': 1}, [799, 201]]]
99
[843, [{'<=50K': 0, '>50K': 1}, [835, 165]]]
999
[800, [{'<=50K': 0, '>50K': 1}, [948, 52]]]


[[0.768, [{'<=50K': 0, '>50K': 1}, [752, 248]]],
 [0.811, [{'<=50K': 0, '>50K': 1}, [775, 225]]],
 [0.819, [{'<=50K': 0, '>50K': 1}, [787, 213]]],
 [0.823, [{'<=50K': 0, '>50K': 1}, [795, 205]]],
 [0.825, [{'<=50K': 0, '>50K': 1}, [799, 201]]],
 [0.843, [{'<=50K': 0, '>50K': 1}, [835, 165]]],
 [0.8, [{'<=50K': 0, '>50K': 1}, [948, 52]]]]

In [82]:
e5 = time.time()

In [83]:
e5 - s5

72.25272560119629

In [84]:
s6 = time.time()

In [85]:
error_rate_all(k_set, v_euclidean, train_all)

1
[4924, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]]
3
[4408, [{'<=50K': 0, '>50K': 1}, [3833, 1167]]]
5
[4271, [{'<=50K': 0, '>50K': 1}, [3888, 1112]]]
7
[4242, [{'<=50K': 0, '>50K': 1}, [3921, 1079]]]
9
[4202, [{'<=50K': 0, '>50K': 1}, [3943, 1057]]]
99
[4090, [{'<=50K': 0, '>50K': 1}, [4143, 857]]]
999
[3891, [{'<=50K': 0, '>50K': 1}, [4758, 242]]]


[[0.9848, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]],
 [0.8816, [{'<=50K': 0, '>50K': 1}, [3833, 1167]]],
 [0.8542, [{'<=50K': 0, '>50K': 1}, [3888, 1112]]],
 [0.8484, [{'<=50K': 0, '>50K': 1}, [3921, 1079]]],
 [0.8404, [{'<=50K': 0, '>50K': 1}, [3943, 1057]]],
 [0.818, [{'<=50K': 0, '>50K': 1}, [4143, 857]]],
 [0.7782, [{'<=50K': 0, '>50K': 1}, [4758, 242]]]]

In [86]:
e6 = time.time()

In [87]:
e6 - s6

364.7721281051636

In [88]:
test = binarization('income.test.blind', train[1])

1000, 91, 2


In [89]:
def mass_predict(k_set, method, test_set):
    output = []
    
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
    
    for k in k_set:
        print(k)
        p_arr = None
        predictions = []
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train, method, k, mapping)
            predictions.append(p)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)

            # Update postive rate
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([p_arr])
        output.append(p_arr)
        output.append(predictions)
        
    return output

In [112]:
s7 = time.time()

In [113]:
blind = mass_predict([99], v_manhattan, test)

99
[[{'<=50K': 0, '>50K': 1}, [806, 194]]]


In [114]:
e7 = time.time()

In [115]:
e7 - s7

4.1429736614227295

In [116]:
with open('income.test.predicted', 'w') as out:
    for i, line in enumerate(open('income.test.blind').readlines()):
        out.write(line[:-2] + ', ' + blind[1][i][0] + '\n')